In [ ]:
import cvxpy as cp
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


inputnodes = [0,6]
inputedges = [(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)]

non_member = [3,4,5,6]
source = 0
recievers = [1,2]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)


# f_star = cp.Variable()  


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []

c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]


fia = {(i, a): cp.Variable() for a in edges for i in recievers}

for i in recievers: 
     for a in edges:
          flow_constraints += [fia[i,a] <= c[a]]
          flow_constraints += [fia[i,a] >= 0]


for i in recievers:
    for j in non_member:
        incoming_edges = [(u, v) for u, v in edges if v == int(j)] 
        outcoming_edges = [(u, v) for u, v in edges if u == int(j)] 
        incoming_flow = sum(fia[i, x] for x in incoming_edges)
        outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
        flow_constraints += [incoming_flow == outcoming_flow]
        print(incoming_edges)

    incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  
    
    outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i)) 
    incoming_flow_reciever = sum(fia[i, x] for x in edges if x[1] == int(i))  

    flow_constraints += [incoming_flow_source == 0]
    flow_constraints += [outcoming_flow_reciever == 0]
    equalrate_constraints += [f_star == incoming_flow_reciever]



# for i in recievers:
#      for j in non_member:
#           incoming_edges = G.in_edges(int(j))
#           outcoming_edges = G.out_edges(j)
#           incoming_flow = 0
#           outcoming_flow = 0
#           for x in incoming_edges:
#                incoming_flow = incoming_flow + fia[i,x]
#           for y in outcoming_edges:
#                outcoming_flow = outcoming_flow + fia[i,y]
#           flow_constraints += [incoming_flow == outcoming_flow]

#      incoming_flow_source = sum(fia[i, x] for x in G.in_edges(0))
#      outcoming_flow_reciever = sum(fia[i, x] for x in G.out_edges(i))
#      flow_constraints += [incoming_flow_source == 0]
#      flow_constraints += [outcoming_flow_reciever == 0]


     # incoming_edges_source = G.in_edges(j)
     # incoming_flow_source = 0
     # for x in incoming_edges_source:
     #      incoming_flow_source = incoming_flow_source + fia[i,x]
     # outcoming_edges = G.out_edges(j)    

    


objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)

